In [1]:
import pandas as pd
import os
import re
import json
import gc
import shutil

In [27]:
class AlphaBoosting:
    def __init__(self, root=None, train_csv_url=None, test_csv_url=None, validation_index=None, timestamp=None,
                 label=None, categorical_features=None, numerical_features=None, validation_ratio=0.1, ngram=(1,1),
                 downsampling=1, down_sampling_ratio=None, configuration=None):
        downsampling_flg = False
        down_sampling_ratio_flg = False
        val_flg = False
        if configuration == None:
            # set configurations
            self.ROOT = root
            self.OUTDIR = root + 'output/'
            self.LOGDIR = root + 'log/'
            self.DATADIR = root + 'data/'
            self.train_csv_url = train_csv_url
            self.test_csv_url = test_csv_url
            self.timestamp = timestamp
            self.label = label
            self.categorical_features = categorical_features
            self.numerical_features = numerical_features
            self.downsampling = downsampling
            self.down_sampling_ratio = down_sampling_ratio 
            # read data
            self._read_data()
            if validation_index == None:
                self.validation_index = list(range(int(self.train_len*(1-validation_ratio)), self.train_len))
            else:
                self.validation_index = validation_index
        else:
            with open(configuration, 'r') as infile: file = json.load(infile)
            self.ROOT = file['root']
            self.OUTDIR = file['root'] + 'output/'
            self.LOGDIR = file['root'] + 'log/'
            self.DATADIR = file['root'] + 'data/'
            self.train_csv_url = file['train_csv_url']
            self.test_csv_url = file['test_csv_url']
            # read data
            self._read_data()
            self.timestamp = file['timestamp']
            self.label = file['label']
            self.categorical_features = file['categorical_features']
            self.numerical_features = file['numerical_features']
            self.validation_index = file['validation_index']
            self.downsampling = file['downsampling']
            self.down_sampling_ratio = file['down_sampling_ratio'] 
            
            # check if validation and down sampling need to be redone
            old_down_sampling = self.downsampling
            if downsampling != self.downsampling:
                downsampling_flg = True
                self.downsampling = downsampling
            if down_sampling_ratio != None and self.down_sampling_ratio != down_sampling_ratio:
                down_sampling_ratio_flg = True
                self.down_sampling_ratio = down_sampling_ratio
            if validation_index != None and self.validation_index != validation_index:
                val_flg = True
                self.validation_index = validation_index
        
        # biuld relavent directories
        self.FEATUREDIR = self.DATADIR + 'features/'
        if not os.path.exists(self.OUTDIR): os.makedirs(self.OUTDIR)
        if not os.path.exists(self.LOGDIR): os.makedirs(self.LOGDIR)
        if not os.path.exists(self.DATADIR): os.makedirs(self.DATADIR)
        if not os.path.exists(self.FEATUREDIR): os.makedirs(self.FEATUREDIR)
            
        # save configuration: c
        print('save config')
        self._save_config(self.LOGDIR + 'config.json')
        
        # generate todo list: c
        print('generate todo list')
        dictionary = self._generate_todo_list()
        
        if downsampling_flg or down_sampling_ratio_flg or val_flg:
            dictionary['val_downsample_generate_index'] = False
            dictionary['val_downsample_split'] = False
            dictionary['val_downsample_generation'] = False
            
            if os.path.exists(self.LOGDIR + 'down_sampling_idx.json'): os.remove(self.LOGDIR + 'down_sampling_idx.json')
            if os.path.exists(self.LOGDIR + 'val.ftr'): os.remove(self.DATADIR + 'val.ftr')
            for i in range(old_down_sampling): 
                if os.path.exists(self.DATADIR + str(i) + '.ftr'):
                    os.remove(self.DATADIR + str(i) + '.ftr')
            shutil.rmtree(self.DATADIR + 'split/')
        
        # feature engineering
        print('feature engineering')
        self._feature_engineering(dictionary)
        
        # get validation
        print('validation')
        self._validation_and_down_sampling(dictionary)
        
        # concatenant test: c
        print('concat test')
        self._concat_test(dictionary)
        
        # grid search
        print('grid search')
        self._grid_search(dictionary)
    
    
    ######### util functions #########
    def _read_data(self):
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        if self.train_csv_url != None: self.train = pd.read_csv(self.train_csv_url)
        if self.test_csv_url != None: self.test = pd.read_csv(self.test_csv_url)
        self.df = pd.concat([self.train, self.test], ignore_index=True)
        self.train_len = self.train.shape[0]
        
    def _renew_status(self, dictionary, key, file_url):
        dictionary[key] = True
        with open(file_url, 'w') as file:
            json.dump(dictionary, file)
            
    def _save_config(self, url):
        d = {
            'root':                 self.ROOT,
            'train_csv_url':        self.train_csv_url,
            'test_csv_url':         self.test_csv_url,
            'timestamp':            self.timestamp,
            'label':                self.label,
            'categorical_features': self.categorical_features,
            'numerical_features':   self.numerical_features,
            'validation_index':     self.validation_index,
            'downsampling':         self.downsampling,
            'down_sampling_ratio':  self.down_sampling_ratio
        }
        with open(url, 'w') as file: json.dump(d, file)
        del d
        gc.collect()
            
    def _get_file_concat(self, base_df, split_folder, concat_folder, is_train, file_name_body):
        prefix = 'train' if is_train else 'test'
        file_name = file_name_body + '.ftr'
        for file in os.listdir(split_folder):
            inter_split = file.split('.')
            if inter_split[-1] == 'ftr':
                splitted = inter_split[0].split('__')
                if splitted[0] == prefix:
                    tmp_ftr = pd.read_feather(split_folder + file)
                    base_df['__'.join(splitted[1:])] = tmp_ftr
                    del tmp_ftr
                    gc.collect()
        base_df.reset_index(drop=True).to_feather(concat_folder + file_name)
        del base_df
        gc.collect()
            
            
    
    ######### main functions #########
    def _generate_todo_list(self):
        if os.path.exists(self.LOGDIR + 'todo_list.json'):
            with open(self.LOGDIR + 'todo_list.json', 'r') as file:
                dictionary = json.load(file)
        else:
            dictionary = {'feature_engineering':           False, 
                          'val_downsample_generate_index': False,
                          'val_downsample_split':          (self.downsampling==0),
                          'val_downsample_generation':     False,
                          'concat_test':                   False,
                          'grid_search':                   False}
            with open(self.LOGDIR + 'todo_list.json', 'w') as file: json.dump(dictionary, file)
        return dictionary
    
    def _feature_engineering(self, dictionary):
        self.train[['b']].to_feather(self.FEATUREDIR + 'train__d.ftr')
        self.train[['a']].to_feather(self.FEATUREDIR + 'train__e.ftr')
        feature_engineering_file_url = self.LOGDIR + 'feature_engineering.txt'
        if not dictionary['feature_engineering']:
            if not os.path.exists(feature_engineering_file_url):
                self._generate_feature_engineering_file(feature_engineering_file_url)
            with open(feature_engineering_file_url, 'r') as file:
                line = file.readline()
                while line:
                    line = re.sub('\\n', '', line)
                    self._add_column(line)
                    # TODO
                    # feature engineering: create feature line by line in the file
        self._renew_status(dictionary, 'feature_engineering', (self.LOGDIR + 'todo_list.json'))
    
    def _validation_and_down_sampling(self, dictionary):
        split_folder = []
        index = []
        down_sampling_url = None
        if not dictionary['val_downsample_generation']:
            # down sampling
            down_sampling_url = self.DATADIR + 'split/'
            if not os.path.exists(down_sampling_url): os.makedirs(down_sampling_url)
            index.extend(self._generate_down_sampling_index_file(dictionary['val_downsample_generate_index']))
            for i in range(self.downsampling): 
                split_folder.append(down_sampling_url+str(i)+'/')
                if not os.path.exists(split_folder[-1]): os.makedirs(split_folder[-1])

            # validation
            split_folder.append(self.DATADIR + 'split/val/')
            index.append(self.validation_index)
            if not os.path.exists(split_folder[-1]): os.makedirs(split_folder[-1])
                
            self._renew_status(dictionary, 'val_downsample_generate_index', self.LOGDIR + 'todo_list.json')
        
        # split files
        if not dictionary['val_downsample_split']:
            for file in os.listdir(self.FEATUREDIR):
                split_file = file.split('.')
                if split_file[-1] == 'ftr':
                    splitted = split_file[0].split('__')
                    if splitted[0] == 'train':
                        for i in range(len(index)):
                            if not os.path.exists(split_folder[i] + file):
                                tmp_df = pd.read_feather(self.FEATUREDIR + file)
                                tmp_df.loc[index[i]].reset_index(drop=True).to_feather(split_folder[i] + file)
                                del tmp_df
                                gc.collect()
            self._renew_status(dictionary, 'val_downsample_split', self.LOGDIR + 'todo_list.json')
        
        # concat files
        if not dictionary['val_downsample_generation']:
            if down_sampling_url == None:
                index.append(sorted(list(set(range(self.train_len)).difference(set(self.validation_index)))))
                split_folder.append(self.FEATUREDIR)
            for i in range(len(split_folder)):
                file_name_body = 'val' if split_folder[i] == self.FEATUREDIR else split_folder[i].split('/')[-2]
                self._get_file_concat(base_df=self.train.loc[index[i]].copy().reset_index(drop=True),
                                      split_folder=split_folder[i], 
                                      concat_folder=self.DATADIR, 
                                      is_train=True, 
                                      file_name_body=file_name_body)
            self._renew_status(dictionary, 'val_downsample_generation', self.LOGDIR + 'todo_list.json')
        
    
    def _grid_search(self, dictionary):
        # TODO: finish grid search
        self._renew_status(dictionary, 'grid_search', self.LOGDIR + 'todo_list.json')
    
    
    ######### support functions #########
    # feature engineering
    def _generate_feature_engineering_file(self, feature_engineering_file_url):
        with open(feature_engineering_file_url, 'w') as file:
            #TODO
            pass
    
    # create a feature
    def _add_column(self):
        #TODO
        pass
    
    # concat test
    def _concat_test(self, dictionary):
        if not dictionary['concat_test']:
            self._get_file_concat(base_df=self.test.copy(), 
                                  split_folder=self.FEATUREDIR, 
                                  concat_folder=self.DATADIR, 
                                  is_train=False, 
                                  file_name_body='test')
            self._renew_status(dictionary, 'concat_test', self.LOGDIR + 'todo_list.json')
        gc.collect()
    
    def _generate_down_sampling_index_file(self, has_file_built):
        
        def _downsampling(positive_idx, negative_idx, ratio):
            idx = np.random.choice(negative_idx, int(ratio*len(negative_idx)), replace=False)
            idx = np.concatenate((idx, positive_idx))
            return np.sort(idx).astype(int).tolist()
        
        index = []
        if has_file_built:
            with open(self.LOGDIR + 'down_sampling_idx.json', 'r') as file:
                index = json.load(file)
        else:
            train_exclude_val = self.train.drop(self.validation_index, axis=0)
            positive = list(train_exclude_val[train_exclude_val[self.label]==1].index.values)
            negative = list(train_exclude_val[train_exclude_val[self.label]==0].index.values)
            ratio = len(positive) / len(negative) if self.down_sampling_ratio == None else self.down_sampling_ratio 
            for i in range(self.downsampling): index.append(_downsampling(positive, negative, ratio))
            del train_exclude_val
            gc.collect()
            with open(self.LOGDIR + 'down_sampling_idx.json', 'w') as file: json.dump(index, file)
        return index
    
    # grid search
    def _generate_grid_search_file(self):
        #TODO
        pass
    
    """ 
    feature_engineering todo list
    feature_engineering.txt line: <function_name>__<feature_combination_name>__<possible_param>
    file_name: train__<function_name>__<feature_combination_name>__<possible_param>.ftr
                test__<function_name>__<feature_combination_name>__<possible_param>.ftr
    """

In [33]:
# a = AlphaBoosting(configuration='./log/config.json')
a = AlphaBoosting(root='./', train_csv_url='./a.txt', test_csv_url='./b.txt', validation_index=[1,2], label='c',down_sampling_ratio=0.5, downsampling=2, configuration='./log/config.json')

save config
generate todo list
redownsampling
['val.ftr', 'features', 'test.ftr'] ['config.json', 'todo_list.json', 'feature_engineering.txt']
feature engineering
validation
{'feature_engineering': True, 'val_downsample_generate_index': False, 'val_downsample_split': False, 'val_downsample_generation': False, 'concat_test': True, 'grid_search': True}
concat test
grid search


In [34]:
import pandas as pd
# print(pd.read_feather('./data/features/train__d.ftr'))
# print()
# print(pd.read_feather('./data/features/train__e.ftr'))
# print()
print(pd.read_feather('./data/0.ftr'))
print()
print(pd.read_feather('./data/1.ftr'))
print()
print(pd.read_feather('./data/val.ftr'))
print()

         a   b  c   d        e
0        5   6  0   6        5
1      100  11  0  11      100
2     1011  12  0  12     1011
3       10  14  0  14       10
4  1111111  15  0  15  1111111
5   111111  16  1  16   111111
6    11111  17  1  17    11111
7       11  18  0  18       11
8     1111  19  1  19     1111
9      111  20  1  20      111

        a   b  c   d       e
0       4   5  0   5       4
1       6   7  0   7       6
2       8   9  0   9       8
3       9  10  0  10       9
4    1011  12  0  12    1011
5     101  13  0  13     101
6  111111  16  1  16  111111
7   11111  17  1  17   11111
8    1111  19  1  19    1111
9     111  20  1  20     111

   a  b  c  d  e
0  2  3  0  3  2
1  3  4  0  4  3

